## Æther Composition Priority
| z-index | Group | Item(s) |
| --- | --- | --- |
| 0 | Background | Background color |
| 1 | Background | Tilemap |
| 2 | Interaction | Non-player sprites |
| 3 | Interaction | Player sprite |
| 4 | Interaction | Projectiles |
| 5 | Effects | Visual effects |
| 6 | Overlay | UI |


## Steps:

### Layer 0: Background 
---
- **Goal: Print a solid color background to the terminal display.**
  - [ ] Print a single color to the terminal.
  - [ ] Fills the terminal by default.
  - [ ] Never exceeds terminal bounds
  - [ ] Can be resized smaller than the terminal
  - [ ] Use 256-color extended ANSI codes.
  - [ ] Keep a `term_utils.py` module for common terminal operations.
- **Classes Needed**
  - [x] NyxEntity
  - [x] NyxEntityManager
  - [x] NyxComponentStore
  - [x] NyxComponent (ABC)
  - [x] BackgroundColorComponent
  - [x] NyxSystem (ABC)
  - [x] RendererSystem
  - [x] AetherCompositor
  - [ ] HemeraRenderer

- After a few trials, working up through the composition layers seems the best way to approach the problem. 
- So far so good through the `NyxSystem` class. Next is working on rendering collection.
- Render system collects and sends the prioritized entity dict to Aether

Next: Aether -> Hemera

### Layer 1: Tilemap 
---
- **Goal**
- **Classes Needed**

### Intermediate Goal: Subpixel Rendering
---
- **Goal**
- **Classes Needed**
- 
Implement this early, before lots and lots of math has to be updated.